## Desafio 02. Sobreposição de assinaturas

### Bibliotecas

In [1]:
import pandas as pd

# Módulo de conexão com banco
import sys
sys.path.append('../') 
import funcoesconexao as con

# Desativa warnings
import warnings
warnings.filterwarnings('ignore')

### Carregamento dos Dados

#### Via banco de dados — SQL Server

In [2]:
conexao = con.conectar_bd()

----- Conexão com banco de dados estabelecida com sucesso! -----


In [3]:
# Carregando dataframes
df_assinaturas = con.carregar_dados(conexao, 'assinaturas')

# Fechando conexão
con.fechar_conexao_bd(conexao)

In [4]:
# Exibindo dataframes
display(df_assinaturas.head())

,usuario_id,data_inicio,data_termino
0,1,2019-01-01,2019-01-31
1,2,2019-01-15,2019-01-17
2,3,2019-01-29,2019-02-04
3,4,2019-02-05,2019-02-10
4,5,2019-03-07,None


#### Via arquivo .CSV

In [5]:
# Carregando dataframes
df_assinaturas = con.carregar_dados_csv('assinaturas.csv')

In [6]:
# Exibindo dataframes
display(df_assinaturas.head())

,usuario_id,data_inicio,data_termino
0,1,2019-01-01,2019-01-31
1,2,2019-01-15,2019-01-17
2,3,2019-01-29,2019-02-04
3,4,2019-02-05,2019-02-10
4,5,2019-03-07,NaN


### Solução

In [7]:
def sobreposicao_assinaturas(assinaturas: pd.DataFrame):
    # Convertendo colunas para o tipo datetime
    assinaturas['data_inicio'] = pd.to_datetime(assinaturas['data_inicio'])
    assinaturas['data_termino'] = pd.to_datetime(assinaturas['data_termino'])

    # Filtrando apenas assinaturas concluídas
    assinaturas = assinaturas[assinaturas['data_termino'].notna()]

    # Ordenando dataframe
    assinaturas = assinaturas.sort_values(by='data_inicio')

    # Criando dicionário
    sobreposicao_status = {}

    # Alimenta dicionário
    for index, row in assinaturas.iterrows():
        usuario_id = row['usuario_id']
        data_inicio = row['data_inicio']
        data_termino = row['data_termino']

        # Verificando se alguma assinatura anterior se sobrepõe à assinatura atual
        assinatura_anterior = assinaturas[
            (assinaturas['data_termino'] >= data_inicio) & 
            (assinaturas['usuario_id'] != usuario_id)
        ]

        if not assinatura_anterior.empty:
            sobreposicao_status[usuario_id] = 1
        else:
            sobreposicao_status[usuario_id] = 0

    # Criando dataframe com status da sobreposição
    df_sobreposicao = pd.DataFrame.from_dict(
        sobreposicao_status, orient='index', columns=['sobreposicao']
    )

    # Renomeando e resetando índice
    df_sobreposicao.index.name = 'usuario_id'
    df_sobreposicao.reset_index(inplace=True)

    # Saída
    return df_sobreposicao

In [8]:
sobreposicao_assinaturas(df_assinaturas)

,usuario_id,sobreposicao
0,1,1
1,2,1
2,3,1
3,4,0
